In [1]:
import os
import random
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm import tqdm

# Set the seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Define a custom dataset
class MovieReviewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        content = row['Content']
        sentiment = row['Sentiment']
        label = 1 if sentiment == 'Positive' else 0

        encoding = self.tokenizer.encode_plus(
            content,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)  # Ensure the label is a long for CrossEntropyLoss
        }

# Load the dataset from CSV files
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')
test_df = pd.read_csv('test_data.csv')

print(train_df.head())

train_indices = train_df.sample(frac=1, random_state=200).index
train_df = train_df.loc[train_indices]

print(train_df.head())

# Create data loaders
max_len = 512
batch_size = 16

train_dataset = MovieReviewsDataset(train_df, tokenizer, max_len)
val_dataset = MovieReviewsDataset(val_df, tokenizer, max_len)
test_dataset = MovieReviewsDataset(test_df, tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



                                             Content Sentiment
0  synopsis : the president of a company wants to...  Negative
1  " take a number , fill out a form , and wait y...  Positive
2  here 's a concept -- jean - claude van damme g...  Negative
3  originally launched in 1978 , this popular fil...  Positive
4  it shows that america remains ambivalent over ...  Positive
                                                Content Sentiment
366   susan granger 's review of " osmosis jones " (...  Positive
1325  for a movie about disco - era excess , " 54 " ...  Positive
133   kate ( jennifer aniston ) is having some probl...  Negative
1419  i 'm not quite sure what to say about mars att...  Positive
1258  saw an advanced screening of the movie sniper ...  Negative


In [2]:
# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, hidden_dim, output_dim, n_layers, dropout_prob):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(bert_model.config.hidden_size, hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout_prob, bidirectional=False)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)
        self.softmax = nn.Softmax(dim=1)  # Add a softmax activation

    def forward(self, bert_embeddings):
        packed_output, (hidden, cell) = self.rnn(bert_embeddings)
        output = self.fc(self.dropout(hidden[-1]))
        return self.softmax(output)  # Apply softmax activation

# Instantiate the model
hidden_dim = 128
output_dim = 2  # Two outputs, one for each class
n_layers = 1
dropout_prob = 0.3

model = RNNModel(hidden_dim, output_dim, n_layers, dropout_prob)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00033960197625562883)

# Training function
def train_epoch(model, data_loader, criterion, optimizer, device, bert_model):
    model.train()
    losses = []
    correct_predictions = 0
    all_preds = []
    all_labels = []

    for batch in tqdm(data_loader, desc="Training", leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)  # Ensure labels are long for CrossEntropyLoss

        # Get BERT embeddings
        with torch.no_grad():
            bert_outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            bert_embeddings = bert_outputs.last_hidden_state

        optimizer.zero_grad()
        outputs = model(bert_embeddings)
        loss = criterion(outputs, labels)
        losses.append(loss.item())
        preds = torch.argmax(outputs, dim=1)
        correct_predictions += torch.sum(preds == labels)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        loss.backward()
        optimizer.step()

    conf_matrix = confusion_matrix(all_labels, all_preds)
    accuracy = correct_predictions.double() / len(data_loader.dataset)
    avg_loss = sum(losses) / len(losses)
    return accuracy, avg_loss, conf_matrix

# Evaluation function
def eval_model(model, data_loader, criterion, device, bert_model):
    model.eval()
    losses = []
    correct_predictions = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)  # Ensure labels are long for CrossEntropyLoss

            # Get BERT embeddings
            bert_outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            bert_embeddings = bert_outputs.last_hidden_state

            outputs = model(bert_embeddings)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            preds = torch.argmax(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    conf_matrix = confusion_matrix(all_labels, all_preds)
    accuracy = correct_predictions.double() / len(data_loader.dataset)
    avg_loss = sum(losses) / len(losses)
    return accuracy, avg_loss, conf_matrix

class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_acc = 0.0
        self.early_stop = False

    def __call__(self, train_acc, model):
        if train_acc > self.best_acc:
            self.best_acc = train_acc
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

# Training loop with early stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
bert_model = bert_model.to(device)  # Move BERT model to device
num_epochs = 40
best_val_acc = 0.0  # Initialize the best validation accuracy to zero
early_stopping = EarlyStopping(patience=5, verbose=True)  # Initialize early stopping

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    train_acc, train_loss, train_conf_matrix = train_epoch(model, train_loader, criterion, optimizer, device, bert_model)
    print(f'Train loss {train_loss:.4f} accuracy {train_acc:.4f}')
    print('Train Confusion Matrix:')
    print(train_conf_matrix)

    val_acc, val_loss, val_conf_matrix = eval_model(model, val_loader, criterion, device, bert_model)
    print(f'Val loss {val_loss:.4f} accuracy {val_acc:.4f}')
    print('Validation Confusion Matrix:')
    print(val_conf_matrix)

    # Save the model if the validation accuracy is the best we've seen so far
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Saved best model at epoch {epoch + 1}')

    # Check early stopping
    early_stopping(train_acc, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Save the model at the last epoch
torch.save(model.state_dict(), 'last_epoch_model.pth')

# Load the best model before evaluating on the test set
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on test data using the best model
test_acc_best, test_loss_best, test_conf_matrix_best = eval_model(model, test_loader, criterion, device, bert_model)
print(f'Best Model Test loss {test_loss_best:.4f} accuracy {test_acc_best:.4f}')
print('Best Model Test Confusion Matrix:')
print(test_conf_matrix_best)

# Load the model from the last epoch
model.load_state_dict(torch.load('last_epoch_model.pth'))

# Evaluate on test data using the last epoch model
test_acc_last, test_loss_last, test_conf_matrix_last = eval_model(model, test_loader, criterion, device, bert_model)
print(f'Last Epoch Model Test loss {test_loss_last:.4f} accuracy {test_acc_last:.4f}')
print('Last Epoch Model Test Confusion Matrix:')
print(test_conf_matrix_last)

/Applications/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/40
----------


Train loss 0.6735 accuracy 0.5881
Train Confusion Matrix:
[[437 363]
 [296 504]]


Val loss 0.6071 accuracy 0.7500
Validation Confusion Matrix:
[[90 10]
 [40 60]]
Saved best model at epoch 1
Epoch 2/40
----------


Train loss 0.6081 accuracy 0.6850
Train Confusion Matrix:
[[558 242]
 [262 538]]


...

Val loss 0.6145 accuracy 0.6950
Validation Confusion Matrix:
[[67 33]
 [28 72]]
EarlyStopping counter: 4 out of 5
Epoch 34/40
----------


Train loss 0.3265 accuracy 0.9869
Train Confusion Matrix:
[[792   8]
 [ 13 787]]


Val loss 0.6140 accuracy 0.6900
Validation Confusion Matrix:
[[67 33]
 [29 71]]
EarlyStopping counter: 5 out of 5
Early stopping


Best Model Test loss 0.5453 accuracy 0.7300
Best Model Test Confusion Matrix:
[[89 11]
 [43 57]]


Last Epoch Model Test loss 0.5403 accuracy 0.7650
Last Epoch Model Test Confusion Matrix:
[[82 18]
 [29 71]]


In [3]:
import torch
from transformers import BertTokenizer, BertModel

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
model = RNNModel(hidden_dim, output_dim, n_layers, dropout_prob)
model.load_state_dict(torch.load('last_epoch_model.pth'))
model.eval()

# Inference function
def predict(text, tokenizer, bert_model, model, device):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        bert_outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        bert_embeddings = bert_outputs.last_hidden_state

        outputs = model(bert_embeddings)
        _, prediction = torch.max(outputs, dim=1)

    return prediction.item()

# Perform inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
bert_model = bert_model.to(device)

text = "That movie is terrific"
prediction = predict(text, tokenizer, bert_model, model, device)

# Map prediction to sentiment
sentiment = "Positive" if prediction == 1 else "Negative"
print(f'Text: "{text}"')
print(f'Predicted Sentiment: {sentiment}')


/Applications/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Text: "That movie is terrific"
Predicted Sentiment: Positive
